# Load and Create Dataset

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Setup Google Cloud BigQuery API

# Replace with your own key path
key_path = "/Users/theodoreleeiv/Documents/Documents - Theodore’s MacBook Pro/Google Cloud/inbound-density-428819-p8-aef577a5f38e.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client()


def QUERY(query):
    """
    Queries BigQuery
    :param query: string
    :return: dataframe
    """
    query = str(query)
    query_job = client.query(query)
    return query_job.result().to_dataframe()

In [11]:
df = QUERY("""
  SELECT
    r.subject_id,
    r.stay_id,
    r.hadm_id,
    r.readmit_stay_id,
    EXTRACT(DAY FROM edadmitdate) AS admit_day,
    EXTRACT(MONTH FROM edadmitdate) AS admit_month,
    EXTRACT(YEAR FROM edadmitdate) AS admit_year,
    EXTRACT(DAY FROM eddischargedate) AS discharge_day,
    EXTRACT(MONTH FROM edadmitdate) AS discharge_month,
    EXTRACT(YEAR FROM edadmitdate) AS discharge_year,
    edadmithour,
    eddischargehour,
    LOS_hr,
    days_to_readmission,
    had_readmission_within_30,
    is_readmission,
    arrival_transport,
    disposition,
    gender,
    anchor_age,
    insurance,
    marital_status,
    language,
    age_group,
    race,
    dod,
    CASE
    WHEN mental_health_category = "Substance-Related Mental Disorders" THEN 1
    ELSE 0
  END AS substance_related_disorder
  

FROM
  `inbound-density-428819-p8.patient_info.MH-ED-READMISSIONS` AS r

LEFT JOIN
  `inbound-density-428819-p8.patient_info.MH-PATIENT-DEMO` AS p

ON r.subject_id = p.subject_id

LEFT JOIN
  `inbound-density-428819-p8.patient_info.MH-ED-DIAGNOSIS` AS d
ON
  r.stay_id = d.stay_id
  
WHERE d.is_mh_disorder = 1

""")

In [12]:
df.head()

,subject_id,stay_id,hadm_id,readmit_stay_id,admit_day,admit_month,admit_year,discharge_day,discharge_month,discharge_year,...,disposition,gender,anchor_age,insurance,marital_status,language,age_group,race,dod,substance_related_disorder
0,10019861,32695851,<NA>,<NA>,11,6,2149,11,6,2149,...,HOME,M,24,None,None,None,20-29,ASIAN,NaT,1
1,10039250,33721578,<NA>,<NA>,18,12,2130,18,12,2130,...,HOME,M,20,None,None,None,20-29,ASIAN,NaT,1
2,10054496,32576559,26173541,<NA>,9,5,2122,10,5,2122,...,HOME,M,18,Private,SINGLE,English,10-19,ASIAN,NaT,0
3,10054496,32890846,<NA>,<NA>,22,5,2124,23,5,2124,...,HOME,M,18,Private,SINGLE,English,10-19,ASIAN,NaT,1
4,10068833,33586715,<NA>,<NA>,18,9,2178,18,9,2178,...,HOME,M,25,None,None,None,20-29,ASIAN,NaT,1


In [ ]:
  # AVG(days_to_readmission) AS avg_readmissions,
  # SUM(had_readmission_within_30) AS total_readmissions,
  # ROUND((SUM(had_readmission_within_30)/COUNT(*))*100,2) AS percent_total_readmissions,
  
  

In [27]:
triage_df = QUERY("""
SELECT
  d.stay_id,
  CASE
    WHEN mental_health_category = "Substance-Related Mental Disorders" THEN 1
    ELSE 0
  END AS substance_related_disorder,
  t.*,
  m.*

FROM
  `inbound-density-428819-p8.patient_info.MH-ED-DIAGNOSIS` AS d

LEFT JOIN
    `physionet-data.mimiciv_ed.triage` AS t
ON 
    d.stay_id = t.stay_id
LEFT JOIN 
    `physionet-data.mimiciv_ed.medrecon` AS m
ON  
    d.stay_id = m.stay_id
WHERE d.is_mh_disorder = 1

""")

In [29]:
triage_df.drop(['stay_id_1','chiefcomplaint','etcdescription','subject_id_1','stay_id_2','charttime','gsn','ndc','etc_rn','etccode'],axis=1,inplace=True)
triage_df

,stay_id,substance_related_disorder,subject_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,name
0,32189108,0,10094706,96.900000000,94.000000000,18.000000000,100.000000000,144.000000000,84.000000000,0,2.000000000,lorazepam
1,32189108,0,10094706,96.900000000,94.000000000,18.000000000,100.000000000,144.000000000,84.000000000,0,2.000000000,lithium carbonate
2,32189108,0,10094706,96.900000000,94.000000000,18.000000000,100.000000000,144.000000000,84.000000000,0,2.000000000,lorazepam
3,32189108,0,10094706,96.900000000,94.000000000,18.000000000,100.000000000,144.000000000,84.000000000,0,2.000000000,Wellbutrin
4,32189108,0,10094706,96.900000000,94.000000000,18.000000000,100.000000000,144.000000000,84.000000000,0,2.000000000,trazodone
...,...,...,...,...,...,...,...,...,...,...,...,...
178258,35088350,0,19093803,97.800000000,116.000000000,20.000000000,95.000000000,149.000000000,74.000000000,0,2.000000000,clonidine
178259,35088350,0,19093803,97.800000000,116.000000000,20.000000000,95.000000000,149.000000000,74.000000000,0,2.000000000,Wellbutrin SR
178260,35088350,0,19093803,97.800000000,116.000000000,20.000000000,95.000000000,149.000000000,74.000000000,0,2.000000000,Seroquel
178261,35088350,0,19093803,97.800000000,116.000000000,20.000000000,95.000000000,149.000000000,74.000000000,0,2.000000000,Neurontin


In [32]:
triage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178263 entries, 0 to 178262
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   stay_id                     178263 non-null  Int64 
 1   substance_related_disorder  178263 non-null  Int64 
 2   subject_id                  178263 non-null  Int64 
 3   temperature                 171272 non-null  object
 4   heartrate                   173704 non-null  object
 5   resprate                    172176 non-null  object
 6   o2sat                       172532 non-null  object
 7   sbp                         173219 non-null  object
 8   dbp                         172815 non-null  object
 9   pain                        174072 non-null  object
 10  acuity                      177256 non-null  object
 11  name                        158221 non-null  object
dtypes: Int64(3), object(9)
memory usage: 16.8+ MB


In [35]:
# change the dtype of temperature, heartrate, reprate,o2sat, sbp, dbp, pain, and acuity to float values

<IntegerArray>
[32189108, 38612303, 38212963, 34554710, 38452509, 38817288, 33986024,
 33940585, 34245097, 38493063,
 ...
 39841134, 36295864, 33701823, 37443662, 32288329, 34712344, 39100049,
 31096511, 37820016, 39869599]
Length: 36388, dtype: Int64
<IntegerArray>
[0, 1]
Length: 2, dtype: Int64
<IntegerArray>
[10094706, 10569459, 12010912, 12138301, 12416836, 12736932, 12951922,
 13179215, 13258401, 13306568,
 ...
 10154663, 15860960, 19872358, 13843625, 10902714, 10281090, 10952151,
 17333829, 17836387, 14166752]
Length: 20709, dtype: Int64
[Decimal('96.900000000') Decimal('99.200000000') Decimal('98.300000000')
 Decimal('97.800000000') Decimal('98.500000000') Decimal('98.400000000')
 Decimal('98.700000000') Decimal('98.100000000') Decimal('96.600000000')
 Decimal('98.000000000') None Decimal('98.200000000')
 Decimal('97.300000000') Decimal('99.700000000') Decimal('99.000000000')
 Decimal('97.000000000') Decimal('98.900000000') Decimal('97.400000000')
 Decimal('98.800000000') Decima